In [1]:
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from xppbe.Model.Solutions_utils import Solution_utils
from xppbe.Model.PDE_Model import PBE
from xppbe import xppbe_path

file_path = os.getcwd()

In [9]:

molecule = 'sphere_+1_D'


Rs = {'born_ion': 1,
      'methanol': 2.54233,
      'methanol_E': 2.54233,
      'arg': 5.9695745,
      '1ubq': 24.697,
      'sphere': 1.2,
      'sphere_+1-1': 1.2,
      'sphere_+1+1': 1.2,
      'sphere_+2-1': 1.2,
      'sphere_+1_D': 1.2,
      'sphere_Nqs': 1.2}

Model = Solution_utils()

Model.main_path = xppbe_path
Model.molecule_path = os.path.join(xppbe_path,'Molecules',molecule)
Model.DTYPE = 'float32'

Model.domain_properties = {
        'molecule': molecule,
        'epsilon_1':  2,
        'epsilon_2': 80,
        'kappa': 0.125,
        'T' : 300 
        }

for key in Model.domain_properties:
      setattr(Model, key, Model.domain_properties[key])

Model.DTYPE = 'float32'
Model.adim = 'kb_T_qe' #'qe_eps0_angs'
class Mesh:
      pass
mesh = Mesh()
mesh.G_sigma = 1
Model.mesh = mesh
Model.fields = ['react','phi']
PBE.calculate_properties(Model,Model.domain_properties)
PBE.get_charges(Model)

Model.equation = 'linear'


-6.021156244466218

In [3]:
Model.q_list

[Charges(q=0.7214, x_q=array([ 0.392,  0.877, -2.219]), r_q=1.908, atom_name='C', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=-0.6013, x_q=array([ 1.262,  0.206, -2.769]), r_q=1.6612, atom_name='O', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 0.586,  1.668, -3.938]), r_q=0.6, atom_name='H', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 0.459,  3.157, -3.268]), r_q=0.6, atom_name='H2', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 1.672,  2.154, -2.813]), r_q=0.6, atom_name='H3', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.1305, x_q=array([-0.126, -1.073, -0.834]), r_q=1.824, atom_name='N', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 0.149, -1.142,  0.125]), r_q=0.6, atom_name='H', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=-0.0223, x_q=array([-0.486,  0.311, -1.129]), r_q=1.908, atom_name='CA', res

In [4]:
dR = 0.4
R_mol = Rs[molecule]
Rmin = dR + R_mol
Rmax = Rmin + 3

Model.DTYPE = 'float32'

### Calculate Solvation energy

In [5]:
phi_q = Model.Spherical_Harmonics(Model.x_qs,'molecule',R_mol, N=30)
print(phi_q)
PBE.solvation_energy_phi_qs(Model,phi_q)

[-0.0262375  -0.07895157  0.00366047 -0.05481606 -0.00050707]


-548.7745934585375

### Create data_known file

In [7]:

def calculate_print_phi(domain,method, num_points, file):

    R = R_mol if domain == 'molecule' else Rmax if domain == 'solvent' else None
    x = np.random.uniform(-R, R, num_points)
    y = np.random.uniform(-R, R, num_points)
    z = np.random.uniform(-R, R, num_points)
    r = np.sqrt(x**2+y**2+z**2)
    X = np.stack([x,y,z], axis=1).astype(np.float32)

    phi_values = Model.phi_known(method,'phi',X,domain,R_mol)

    if domain == 'molecule':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]<R_mol:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

    elif domain == 'solvent':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]>Rmin and r[i]<Rmax:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

In [11]:
with open(os.path.join(file_path,molecule,'data_known.dat'), 'w') as file:
    calculate_print_phi('solvent','G_Yukawa',2500, file)
    # calculate_print_phi('molecule','Harmonic_spheres',300, file)

print('File created')

File created


### Phi ENS

In [1]:
from xppbe.Post.Postcode import Postprocessing as Post_CLS
from xppbe.Mesh.Mesh import Domain_Mesh
from xppbe.Model.Solutions_utils import Solution_utils
from xppbe.Model.PDE_Model import PBE
from xppbe import xppbe_path
import os
import tensorflow as tf

Model = Solution_utils()

molecule = 'born_ion'

results_path = os.path.join(os.getcwd(),'results')
Model.results_path = results_path
Model.main_path = xppbe_path
Model.molecule_path = os.path.join(xppbe_path,'Molecules',molecule)


Model.domain_properties = {
        'molecule': molecule,
        'epsilon_1':  2,
        'epsilon_2': 80,
        'kappa': 0.125,
        'T' : 300 
        }

for key in Model.domain_properties:
      setattr(Model, key, Model.domain_properties[key])

Model.DTYPE = 'float32'
PBE.get_charges(Model)
Model.aprox_exp = PBE.aprox_exp


mesh_properties = {
        'vol_max_interior': 0.05,
        'vol_max_exterior': 0.6,
        'density_mol': 11.0,
        'density_border': 0.2,
        'dx_experimental': 0.5,
        'mesh_generator': 'msms',
        'dR_exterior': 6.0
        }

Mol_mesh = Domain_Mesh(Model.domain_properties['molecule'], 
                mesh_properties=mesh_properties, 
                save_points=True,
                path=xppbe_path,
                simulation='phi_ens',
                results_path=results_path
                )

Post = Post_CLS(XPINN='', solution_utils=True)
Post.PDE = Model
Post.mesh = Mol_mesh
Model.mesh = Mol_mesh
Model.DTYPE = 'float32'

Mesh initialization ready


In [2]:
meshes_domain = dict()
meshes_domain['E2'] = {'domain': 'solvent', 'type': 'E2', 'file': None,'method':'mesh'}
Post.mesh.adapt_meshes_domain(meshes_domain,Post.PDE.q_list)

((X,_),flag,_)  = Post.mesh.domain_mesh_data['E2']


In [6]:
X_q = list()
for q in Model.q_list:
    if q.atom_name in ['H','I']:
        xq = tf.reshape(tf.constant(q.x_q, dtype=Model.DTYPE), (1,3))
        r_q = tf.constant(q.r_q, dtype=Model.DTYPE)
        X_q.append((xq,r_q))

In [7]:
with open(os.path.join(Model.molecule_path,'data_experimental.dat'),'w') as file:
    methods = ['mean','exponential']
    for method in methods:
        ENS = PBE.get_phi_ens(Model,'Post.model',(X,flag),X_q, method=method, pinn=False, known_method='analytic_Born_Ion')
        file.write(f'## {method} ##\n')
        for j in range(len(X_q)):
            file.write(f'{str(j+1)} {str(ENS[j].numpy())}\n')
        file.write('\n')
        